# Generate Noun Property Ratings (Category-Based)

This notebook rates all nouns on their category-specific properties and saves to JSON/JSONL.

**Input:**
- `categories_tree.json` - Nouns organized by category
- `category_properties.json` - Properties for each category

**Output:** 
- `noun_property_ratings.jsonl` - JSONL format (one JSON object per line)
- `noun_property_nested.json` - Nested JSON format (nouns with property objects)

## 1. Setup

In [1]:
import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from tqdm import tqdm
import os
import re
import json

/Users/b/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## 2. Configuration

In [2]:
# File paths
CATEGORIES_FILE = "categories_tree.json"
PROPERTIES_FILE = "category_properties.json"
OUTPUT_FILE = "grouped_noun_property_ratings.jsonl"  # JSONL format (one JSON object per line)

# How often to save progress
SAVE_FREQUENCY = 200

## 3. Load Model

In [3]:
print("Loading model...")

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="mps",  # Change to "cuda" for NVIDIA GPU or "cpu" for CPU, mps for mac
    torch_dtype="auto",
    trust_remote_code=False,
)

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=50,
    do_sample=False,
)

print("Model loaded!")

Loading model...


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use mps
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Model loaded!


## 4. Load Categories and Properties

In [4]:
# Load categories tree
with open(CATEGORIES_FILE, 'r') as f:
    categories_data = json.load(f)

# Load properties for each category
with open(PROPERTIES_FILE, 'r') as f:
    properties_data = json.load(f)

# Extract noun-to-category mapping
noun_to_category = {}
category_to_nouns = categories_data['Categories']

for category, nouns in category_to_nouns.items():
    for noun in nouns:
        noun_to_category[noun] = category

# Get all unique nouns
all_nouns = list(noun_to_category.keys())

print(f"Loaded {len(all_nouns)} nouns across {len(category_to_nouns)} categories")
print(f"\nCategories:")
for category in category_to_nouns.keys():
    noun_count = len(category_to_nouns[category])
    #prop_count = len(properties_data['category_properties'][category]['properties'])
    prop_count = len(properties_data[category]['properties'])
    print(f"  {category}: {noun_count} nouns, {prop_count} properties")

# Calculate total ratings needed
total_ratings = sum(
    #len(nouns) * len(properties_data['category_properties'][category]['properties'])
    len(nouns) * len(properties_data[category]['properties'])
    for category, nouns in category_to_nouns.items()
)

print(f"\nTotal ratings needed: {total_ratings:,}")
print(f"Estimated time: {total_ratings * 2 / 3600:.1f} hours (at 2 sec/rating)")

Loaded 1852 nouns across 20 categories

Categories:
  Domesticated Mammals: 33 nouns, 10 properties
  Wild Mammals: 72 nouns, 10 properties
  Reptiles & Amphibians: 51 nouns, 10 properties
  Birds, Fish, Insects & Other Animals: 44 nouns, 10 properties
  Trees: 12 nouns, 10 properties
  Non-Tree Plants: 25 nouns, 10 properties
  Fruits: 27 nouns, 10 properties
  Other Raw Foods: 16 nouns, 10 properties
  Prepared Foods & Meals: 51 nouns, 10 properties
  Aircraft: 16 nouns, 10 properties
  Motorcycles: 37 nouns, 10 properties
  Cars, Trucks, Boats & Other Motorized Vehicles: 65 nouns, 10 properties
  Non-Motorized Transportation: 11 nouns, 10 properties
  Geological Features & Minerals: 66 nouns, 10 properties
  Natural Non-Geological Features: 60 nouns, 10 properties
  Buildings & Large Structures: 28 nouns, 10 properties
  Handheld Objects & Tools: 78 nouns, 10 properties
  Transportation Infrastructure: 19 nouns, 10 properties
  Abstract Concepts, Places, People & Activities: 996 nou

## 5. Rating Function

In [5]:
def rate_noun_on_property(noun, property_info, pipe):
    """
    Rate a noun on a property from 1-10.
    Optimized for microsoft/Phi-3-mini-4k-instruct.
    Returns -1 if rating extraction fails.
    """
    property_name = property_info['name']
    description = property_info['description']
    scale = property_info['scale']
    
    # Extract scale endpoints
    scale_parts = scale.split(' to ')
    if len(scale_parts) == 2:
        low_end = scale_parts[0].replace('1 ', '').strip('()')
        high_end = scale_parts[1].replace('10 ', '').strip('()')
    else:
        low_end = "minimum"
        high_end = "maximum"
    
    # Phi-3-mini optimized prompt: short, direct, numbered format
    prompt = f"""Rate "{noun}" for: {property_name}
{description}

Scale:
1={low_end}
10={high_end}

Output only the number 1-10."""
    
    messages = [{"role": "user", "content": prompt}]
    
    try:
        output = pipe(messages)
        response = output[0]["generated_text"].strip()
        
        # Extract first valid number from response
        numbers = re.findall(r'\b([1-9]|10)\b', response)
        if numbers:
            rating = int(numbers[0])
            if 1 <= rating <= 10:
                return rating
        
        return -1
    except Exception as e:
        print(f"Error: {noun} - {property_name}: {str(e)}")
        return -1

## 6. Test Rating Function

In [6]:
# Quick test with a domesticated mammal
print("Testing...")
test_category = "Domesticated Mammals"
test_noun = "elephant" if "elephant" in category_to_nouns.get(test_category, []) else category_to_nouns[test_category][0]
#test_property = properties_data['category_properties'][test_category]['properties'][0]
test_property = properties_data[test_category]['properties'][0]

test_rating = rate_noun_on_property(test_noun, test_property, pipe)
print(f"{test_noun} - {test_property['name']}: {test_rating}/10")
print(f"Property description: {test_property['description']}")
print(f"Scale: {test_property['scale']}")

Testing...
airedale terrier - size: 7/10
Property description: Physical size of the animal
Scale: 1 = very small (hamster, guinea pig), 5 = medium (cat, small dog), 10 = very large (cow, horse)


## 7. Generate All Ratings

In [7]:
# Check for existing progress
if os.path.exists(OUTPUT_FILE):
    existing_ratings = []
    with open(OUTPUT_FILE, 'r') as f:
        for line in f:
            existing_ratings.append(json.loads(line))
    completed = set((r['noun'], r['property']) for r in existing_ratings)
    print(f"Found {len(existing_ratings)} existing ratings")
else:
    existing_ratings = []
    completed = set()
    print("Starting fresh")

# Create list of all work (noun, category, property_info)
all_combinations = []
for category, nouns in category_to_nouns.items():
    #properties = properties_data['category_properties'][category]['properties']
    properties = properties_data[category]['properties']
    for noun in nouns:
        for prop_info in properties:
            all_combinations.append((noun, category, prop_info))

# Filter out completed work
remaining = [
    combo for combo in all_combinations 
    if (combo[0], combo[2]['name']) not in completed
]

print(f"Remaining: {len(remaining):,} ratings")
print("\nStarting...\n")

Found 2000 existing ratings
Remaining: 16,520 ratings

Starting...



In [8]:
# Main loop
results = []

for i, (noun, category, prop_info) in enumerate(tqdm(remaining, desc="Rating")):
    # Get rating
    rating = rate_noun_on_property(noun, prop_info, pipe)
    
    # Store
    results.append({
        'noun': noun,
        'category': category,
        'property': prop_info['name'],
        'rating': rating
    })
    
    # Save periodically
    if (i + 1) % SAVE_FREQUENCY == 0:
        # Append new results to file
        with open(OUTPUT_FILE, 'a') as f:
            for result in results:
                f.write(json.dumps(result) + '\n')
        results = []
        print(f"Saved at {i + 1} ratings")

# Final save
if results:
    with open(OUTPUT_FILE, 'a') as f:
        for result in results:
            f.write(json.dumps(result) + '\n')

print(f"\nDone! Saved to {OUTPUT_FILE}")

Rating:   1%|          | 200/16520 [01:22<1:51:51,  2.43it/s]

Saved at 200 ratings


Rating:   2%|▏         | 400/16520 [02:52<2:15:27,  1.98it/s]

Saved at 400 ratings


Rating:   4%|▎         | 600/16520 [04:29<2:10:26,  2.03it/s]

...

Saved at 16200 ratings


Rating:  99%|█████████▉| 16400/16520 [2:21:47<01:10,  1.70it/s]

Saved at 16400 ratings


Rating: 100%|██████████| 16520/16520 [2:22:53<00:00,  1.93it/s]


Done! Saved to grouped_noun_property_ratings.jsonl


## 8. View Results

In [9]:
# Load and display results
ratings = []
with open(OUTPUT_FILE, 'r') as f:
    for line in f:
        ratings.append(json.loads(line))

print(f"Total ratings: {len(ratings):,}")
failed = sum(1 for r in ratings if r['rating'] == -1)
print(f"Failed ratings: {failed}")
print(f"Success rate: {(len(ratings) - failed) / len(ratings) * 100:.1f}%")

print("\nFirst 20 ratings:")
for rating in ratings[:20]:
    print(rating)

Total ratings: 18,520
Failed ratings: 6
Success rate: 100.0%

First 20 ratings:
{'noun': 'airedale terrier', 'category': 'Domesticated Mammals', 'property': 'size', 'rating': 7}
{'noun': 'airedale terrier', 'category': 'Domesticated Mammals', 'property': 'commonness_as_pet', 'rating': 8}
{'noun': 'airedale terrier', 'category': 'Domesticated Mammals', 'property': 'energy_level', 'rating': 8}
{'noun': 'airedale terrier', 'category': 'Domesticated Mammals', 'property': 'grooming_needs', 'rating': 7}
{'noun': 'airedale terrier', 'category': 'Domesticated Mammals', 'property': 'intelligence_trainability', 'rating': 8}
{'noun': 'airedale terrier', 'category': 'Domesticated Mammals', 'property': 'vocalization_level', 'rating': 7}
{'noun': 'airedale terrier', 'category': 'Domesticated Mammals', 'property': 'lifespan', 'rating': 8}
{'noun': 'airedale terrier', 'category': 'Domesticated Mammals', 'property': 'indoor_suitability', 'rating': 8}
{'noun': 'airedale terrier', 'category': 'Domesticat

In [10]:
# Create nested format (optional - easier to work with)
# Structure: { noun: { category: str, properties: { property: rating, ... } }, ... }
nested = {}
for rating in ratings:
    if rating['rating'] != -1:  # Skip failed ratings
        noun = rating['noun']
        if noun not in nested:
            nested[noun] = {
                'category': rating['category'],
                'properties': {}
            }
        nested[noun]['properties'][rating['property']] = rating['rating']

with open('noun_property_nested.json', 'w') as f:
    json.dump(nested, f, indent=2)

print("Also saved nested format to: noun_property_nested.json")
print(f"Structure: {len(nested)} nouns with category and properties as nested objects")

Also saved nested format to: noun_property_nested.json
Structure: 1852 nouns with category and properties as nested objects


## 9. Summary Statistics by Category

In [11]:
# Show statistics by category
from collections import defaultdict

category_stats = defaultdict(lambda: {'total': 0, 'failed': 0})

for rating in ratings:
    cat = rating['category']
    category_stats[cat]['total'] += 1
    if rating['rating'] == -1:
        category_stats[cat]['failed'] += 1

print("\nStatistics by Category:")
print(f"{'Category':<45} {'Total':<10} {'Failed':<10} {'Success Rate':<15}")
print("="*80)

for cat in sorted(category_stats.keys()):
    stats = category_stats[cat]
    success_rate = (stats['total'] - stats['failed']) / stats['total'] * 100
    print(f"{cat:<45} {stats['total']:<10} {stats['failed']:<10} {success_rate:<15.1f}%")


Statistics by Category:
Category                                      Total      Failed     Success Rate   
Abstract Concepts, Places, People & Activities 9960       2          100.0          %
Aircraft                                      160        0          100.0          %
Birds, Fish, Insects & Other Animals          440        0          100.0          %
Buildings & Large Structures                  280        0          100.0          %
Cars, Trucks, Boats & Other Motorized Vehicles 650        0          100.0          %
Domesticated Mammals                          330        3          99.1           %
Fruits                                        270        0          100.0          %
Geological Features & Minerals                660        0          100.0          %
Handheld Objects & Tools                      780        0          100.0          %
Motorcycles                                   370        1          99.7           %
Natural Non-Geological Features        